# Propositional logic

In [1]:
data PropL = PVar String | PNot PropL | PropL `PAnd` PropL | PropL `POr` PropL deriving Eq

In [2]:
instance Show PropL where
    show (PVar s) = s
    show (PNot p) = "~" ++ show p
    show (p `PAnd` q) = "(" ++ show p ++ " & " ++ show q ++ ")"
    show (p `POr` q) = "(" ++ show p ++ " | " ++ show q ++ ")"

In [3]:
form1 = PVar "p1" `PAnd` PNot (PVar "p1" `POr` PVar "p3")

In [4]:
show form1

"(p1 & ~(p1 | p3))"

In [5]:
form2 = (PVar "p3" `PAnd` PVar "p3") `POr` PNot (PNot (PVar "p1") `POr` PVar "p3")

In [6]:
form2

((p3 & p3) | ~(~p1 | p3))

## Number of operators

In [7]:
opsNr :: PropL -> Int
opsNr (PVar _) = 0
opsNr (PNot p) = 1 + opsNr p
opsNr (PAnd p q) = 1 + opsNr p + opsNr q
opsNr (POr p q) = 1 + opsNr p + opsNr q

In [8]:
opsNr form1

3

```
opsNr (PVar "p1" `PAnd` PNot (PVar "p1" `POr` PVar "p3"))
= 1 + opsNr (PVar "p1) + opsNr (PNot (PVar "p1" `POr` PVar "p3))
= 1 + 0 + opsNr (PNot (PVar "p1" `POr` PVar "p3"))
= 1 + 0 + 1 + opsNr (PVar "p1" `POr` PVar "p3")
= 1 + 0 + 1 + 1 + opsNr (PVar "p1") + opsNr (PVar "p3)
= 1 + 0 + 1 + 1 + 0 + 0
= 3
```

In [9]:
depth :: PropL -> Int
depth (PVar _) = 0
depth (PNot p) = 1 + depth p
depth (PAnd p q) = 1 + max (depth p) (depth q)
depth (POr p q) = 1 + max (depth p) (depth q)

In [10]:
depth form1

3

In [11]:
depth form2

4

In [12]:
gatherNames :: PropL -> [String]                  
gatherNames (PVar s) = [s]                        
gatherNames (PNot p) = gatherNames p                
gatherNames (PAnd p q) = gatherNames p ++ gatherNames q
gatherNames (POr p q) = gatherNames p ++ gatherNames q

In [13]:
gatherNames form2

["p3","p3","p1","p3"]

In [14]:
import Data.List

propNames :: PropL -> [String]
propNames = sort . nub . gatherNames

In [15]:
propNames form2

["p1","p3"]

# Conversion to CNF

## de Morgan's (non recursive)

In [16]:
dM1 :: PropL -> PropL
dM1 (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM1 p = p

In [17]:
dM1 (PNot (PVar "p1" `POr` PVar "p2"))

(~p1 & ~p2)

In [18]:
dM1 form2

((p3 & p3) | ~(~p1 | p3))

In [19]:
dM :: PropL -> PropL
dM (PNot (p `PAnd` q)) = PNot p `POr` PNot q
dM (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM p = p

In [20]:
dM form2

((p3 & p3) | ~(~p1 | p3))

## Recursive de Morgan's

In [21]:
dMrec :: PropL -> PropL
dMrec (PNot (p `PAnd` q)) = PNot (dM p) `POr` PNot (dM q)
dMrec (PNot (p `POr` q)) = PNot (dM p) `PAnd` PNot (dM q)
dMrec (PNot p) = PNot (dM p)
dMrec (p `PAnd` q) = dM p `PAnd` dM q
dMrec (p `POr` q) = dM p `POr` dM q
dMrec (PVar p) = PVar p

In [22]:
form2

((p3 & p3) | ~(~p1 | p3))

In [23]:
dMrec form2

((p3 & p3) | (~~p1 & ~p3))

## Recursive DNE

In [24]:
dne :: PropL -> PropL
dne (PNot (PNot p)) = dne p
dne (PNot p) = PNot (dne p)
dne (p `PAnd` q) = dne p `PAnd` dne q 
dne (p `POr` q) = dne p `POr` dne q 
dne (PVar p) = PVar p

In [25]:
toCNF :: PropL -> PropL
toCNF = dne . dMrec

In [26]:
toCNF form2

((p3 & p3) | (p1 & ~p3))

## Distributive Law

In [27]:
distLaw :: PropL -> PropL 
distLaw ((p `PAnd` q) `POr` (r `PAnd` s)) = (distLaw p `POr` distLaw r) `PAnd` (distLaw p `POr` distLaw s) `PAnd` (distLaw q `POr` distLaw r) `PAnd` (distLaw q `POr` distLaw s) -- double distributivity
distLaw (p `POr` (q `PAnd` r)) = (distLaw p `POr` distLaw q) `PAnd` (distLaw p `POr` r) --left dist
distLaw ((q `PAnd` r) `POr` p) = (distLaw q `POr` distLaw p) `PAnd` (distLaw r `POr` distLaw p) --right dist
distLaw (PNot p) = PNot (distLaw p)
distLaw (p `PAnd` q) = distLaw p `PAnd` distLaw q
distLaw (p `POr` q) = distLaw p `POr` distLaw q
distLaw (PVar p) = PVar p

In [28]:
distLaw form2

((p3 | ~(~p1 | p3)) & (p3 | ~(~p1 | p3)))

In [29]:
toCNF = distLaw . dne . dM

In [30]:
toCNF form2

((p3 | ~(~p1 | p3)) & (p3 | ~(~p1 | p3)))

## Computing truth tables

In [31]:
type VarAssignment = [(String,Bool)]
type TruthTable = [(VarAssignment,Bool)]

Our first step is to gather all of the variables used in a formula and generate all of the possible variable assignments.

In [32]:
propNames form2

["p1","p3"]

In [51]:
{-# LANGUAGE ParallelListComp #-}

import Control.Monad (replicateM)

mkAssignments' :: [String] -> [VarAssignment]
mkAssignments' vs = [[(v,t) | v <- vs | t <- ts ] | ts <- replicateM (length vs) [True,False]]

mkAssignments'' vs = [ zip vs ts | ts <- replicateM (length vs) [True,False]]

In [52]:
exAssigns = mkAssignments'' ["p", "q"]
exAssigns

[[("p",True),("q",True)],[("p",True),("q",False)],[("p",False),("q",True)],[("p",False),("q",False)]]

In [53]:
import Control.Monad (replicateM)

replicateM 3 [True,False]

[[True,True,True],[True,True,False],[True,False,True],[True,False,False],[False,True,True],[False,True,False],[False,False,True],[False,False,False]]

In [54]:
mkAssignments :: PropL -> [VarAssignment]
mkAssignments = mkAssignments' . propNames

In [55]:
lookup :: (Eq a) => [(a,b)] -> a -> b
lookup [] _ = undefined
lookup ((key,val):ps) x = if key == x then val else lookup ps x

In [56]:
mkAssignments form2

[[("p1",True),("p3",True)],[("p1",True),("p3",False)],[("p1",False),("p3",True)],[("p1",False),("p3",False)]]

In [57]:
exAssignments = mkAssignments form2 !! 0

Line 1: Use head
Found:
mkAssignments form2 !! 0
Why not:
head (mkAssignments form2)

In [58]:
exAssignments

[("p1",True),("p3",True)]

In [59]:
lookup exAssignments "p1"

True

In [60]:
interpretAtA :: VarAssignment -> PropL -> Bool
interpretAtA a (PVar v) = lookup a v
interpretAtA a (PNot p) = not (interpretAtA a p)
interpretAtA a (p `PAnd` q) = interpretAtA a p && interpretAtA a q
interpretAtA a (p `POr` q) = interpretAtA a p || interpretAtA a q

In [61]:
interpretAtA exAssignments form2

True

In [62]:
form2

((p3 & p3) | ~(~p1 | p3))

In [63]:
toTruthTable :: PropL -> [(VarAssignment,Bool)]
toTruthTable p = [(a, interpretAtA a p) | a <- mkAssignments p]

In [64]:
toTruthTable form2

[([("p1",True),("p3",True)],True),([("p1",True),("p3",False)],True),([("p1",False),("p3",True)],True),([("p1",False),("p3",False)],False)]

In [65]:
toTruthTable (PNot (PVar "p" `PAnd` PVar "q") `POr` PVar "r")

[([("p",True),("q",True),("r",True)],True),([("p",True),("q",True),("r",False)],False),([("p",True),("q",False),("r",True)],True),([("p",True),("q",False),("r",False)],True),([("p",False),("q",True),("r",True)],True),([("p",False),("q",True),("r",False)],True),([("p",False),("q",False),("r",True)],True),([("p",False),("q",False),("r",False)],True)]

In [66]:
formAnd = PVar "p1" `PAnd` PVar "p2"
exTable = toTruthTable formAnd

## Interpretation using a case expression

In [49]:
interpretAtA a exp = case exp of
    (PVar p) -> lookup a p
    (PNot p) -> not (interpretAtA a p)
    (p `PAnd` q) -> interpretAtA a p && interpretAtA a q
    (p `POr` q) -> interpretAtA a p || interpretAtA a q

In [50]:
gatherNames (PNot (PVar "p" `PAnd` PVar "q") `POr` PVar "r")

["p","q","r"]

In [67]:
show True

"True"

## Pretty printing a truth-table

In [75]:
printBool :: Bool -> String
printBool True = "1"
printBool False = "0"

prettyPrintAssignment :: VarAssignment -> String
prettyPrintAssignment [] = ""
prettyPrintAssignment ((v,t):as) = printBool t ++ " " ++ prettyPrintAssignment as


prettyPrintTable :: [(VarAssignment,Bool)] -> String
prettyPrintTable [] = ""
prettyPrintTable ((a,t):ls) = prettyPrintAssignment a ++ "| " ++ printBool t ++ "\n" ++ prettyPrintTable ls

In [95]:
putStrLn $ prettyPrintTable exTable

1 1 | 1
1 0 | 0
0 1 | 0
0 0 | 0

In [98]:
printPrettyTable :: PropL -> String
printPrettyTable p = (unwords $ propNames p) ++ " | " ++ show p ++ "\n" ++ prettyPrintTable (toTruthTable p)

Line 2: Move brackets to avoid $
Found:
(unwords $ propNames p)
  ++ " | " ++ show p ++ "\n" ++ prettyPrintTable (toTruthTable p)
Why not:
unwords (propNames p)
  ++ " | " ++ show p ++ "\n" ++ prettyPrintTable (toTruthTable p)

In [100]:
putStrLn $ printPrettyTable form2

p1 p3 | ((p3 & p3) | ~(~p1 | p3))
1 1 | 1
1 0 | 1
0 1 | 1
0 0 | 0

In [94]:
propNames form2

["p1","p3"]